<a href="https://colab.research.google.com/github/lhoegler16/Biersortenerkennung/blob/main/Application_Biersortenerkenner_WebBrowser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Classifier zu Biersorten
Folgende Biere werden erkannt:

      *Baumgartner
        *Baumgartner Export
        *Baumgartner Märzen
      *Goesser
        *Goesser Bock
        *Goesser Märzen
        *Gösser Naturgold
      *Guinnes
        *Guinness Draught
        *Guinness Dublin Porter
        *Guinness Golden ALe
        *Guinness Nitro IPA
        *Guinness Special Export
        *Guinness West Indies Porter
      *Rieder
        *Rieder Märzen
        *Rieder Pils
        *Rieder Urecht
        *Rieder Weisse
        *Stiegl
        *Stiegl Bio Zwickl
        *Stiegl Goldbräu
        *Stiegl Stieglbock
        *Stiegl Weisse
      *Wieselburger
        *Wieselburger Gold
        *Wieselburger Schwarzbier
        *Wieselburger Stammbraeu
        *Wiselburger Zwickl
      *Zipfer
        *Zipfer Drei
        *Zipfer Märzen
        *Zipfer Stefanibock
        *Zipfer Urtyp



In [33]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
!pip install fastai --uprade -q



Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: --uprade


In [3]:
!pip install -U fastai

     |████████████████████████████████| 194kB 8.4MB/s 
     |████████████████████████████████| 12.8MB 318kB/s 
     |████████████████████████████████| 61kB 6.5MB/s 
     |████████████████████████████████| 776.8MB 23kB/s 
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.7.1 which is incompatible.
  Found existing installation: torch 1.8.1+cu101
    Uninstalling torch-1.8.1+cu101:
      Successfully uninstalled torch-1.8.1+cu101
  Found existing installation: torchvision 0.9.1+cu101
    Uninstalling torchvision-0.9.1+cu101:
      Successfully uninstalled torchvision-0.9.1+cu101
  Found existing installation: fastai 1.0.61
    Uninstalling fastai-1.0.61:
      Successfully uninstalled fastai-1.0.61


In [35]:
from fastai.vision.all import*

In [36]:
import requests

In [37]:
import ipywidgets as widgets

In [38]:
def download_file_from_google_drive(id, destination):
  URL="https://docs.google.com/uc?export=download"
  session = requests.Session()
  response = session.get(URL, params = {'id' : id},stream = True)
  token = get_confirm_token(response)
  if token:
    params = {'id' : id  , 'confirm' : token}
    response = session.get(URL,params= params, stream = True)
    
def get_confirm_token(response):
  for key, value in response.cookies.items():
    if key.startswith('download_warning'):
      return value

  return None

def save_response_content(response, destination):
  CHUNK_SIZE = 32768
  with open(destination, "wb") as f:
    for chunk in response.iter_content(CHUNK_SIZE):
      if chunk: # filter out keep-alive new chunks
        f.write(chunk)

    

In [39]:
file_id = "1-10QV8OJx9taXBuyI61AOwGrhywPTpn1"
destination = './biersorten_classifier.pkl'
download_file_from_google_drive(file_id,destination)

In [40]:
!cp drive/MyDrive/Biersortenerkenner/biersorten_classifier.pkl ./

In [41]:
path = Path();

inference = load_learner(path/'biersorten_classifier.pkl', cpu=True)

In [43]:
btn_upload = widgets.FileUpload()
lbl_pred = widgets.Label()
btn_run = widgets.Button(description='Classify')
out_p1 = widgets.Output()
def on_click_classify(change):
  img = PILImage.create(btn_upload.data[-1])
  out_p1.clear_output()
  with out_p1: display(img.to_thumb(128,128))
  pred,pred_idx,probs = inference.predict(img)
  lbl_pred.value = f'Prediction: {pred} ; Probability: {probs[pred_idx]:.04f}'

btn_run.on_click(on_click_classify)
widgets.VBox([widgets.Label('Select your beer'),
      btn_upload, btn_run,out_p1,lbl_pred])